In [1]:
%load_ext autoreload
%autoreload 2

import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve,f1_score,balanced_accuracy_score
from answer_judge.vqaEval import VQAEval

In [2]:
data_reader=ReadData('/root/autodl-tmp/hallscope/VQA/train')
data=data_reader.read_all()
random.shuffle(data)

In [3]:
df=pd.DataFrame(data)

In [4]:
df.head()

,img_path,question,answers,question_id,most_likely,responses
0,[/root/autodl-fs/coco_images/train/COCO_train2...,[How many people are riding their bicycle?\nAn...,"[{'answer': ['5', 'expressway', 'no', 'freeway...","[tensor(266108010), tensor(266108011), tensor(...",{'embedding': [[[0.01165771 0.00469971 0.04418...,None
1,[/root/autodl-fs/coco_images/train/COCO_train2...,[What color is the woman's hair?\nAnswer the q...,"[{'answer': ['brown', 'shirt and vest', '1', '...","[tensor(3008001), tensor(3008002), tensor(3008...",{'embedding': [[[ 0.00415039 0.0168457 0.02...,None
2,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is the ocean nearby?\nAnswer the question usi...,"[{'answer': ['no', 'male and female', 'kites',...","[tensor(132838012), tensor(132838013), tensor(...",{'embedding': [[[-0.00189972 0.01818848 -0.00...,None
3,[/root/autodl-fs/coco_images/train/COCO_train2...,[Are the lights on in this room?\nAnswer the q...,"[{'answer': ['no', 'sink', 'yes', 'yes', 'yes'...","[tensor(394036008), tensor(394036009), tensor(...",{'embedding': [[[ 0.01013184 0.00717163 -0.00...,None
4,[/root/autodl-fs/coco_images/train/COCO_train2...,[What is he doing?\nAnswer the question using ...,"[{'answer': ['grinding', 'black', 'sandwich', ...","[tensor(1811002), tensor(1813000), tensor(1813...",{'embedding': [[[ 0.01391602 0.01867676 -0.01...,None


In [5]:
df.shape

(2500, 6)

In [6]:
i=0

In [7]:
df.iloc[i]['img_path']

['/root/autodl-fs/coco_images/train/COCO_train2014_000000266108.jpg',
 '/root/autodl-fs/coco_images/train/COCO_train2014_000000266108.jpg',
 '/root/autodl-fs/coco_images/train/COCO_train2014_000000266108.jpg',
 '/root/autodl-fs/coco_images/train/COCO_train2014_000000266108.jpg',
 '/root/autodl-fs/coco_images/train/COCO_train2014_000000266108.jpg',
 '/root/autodl-fs/coco_images/train/COCO_train2014_000000266108.jpg',
 '/root/autodl-fs/coco_images/train/COCO_train2014_000000266108.jpg',
 '/root/autodl-fs/coco_images/train/COCO_train2014_000000266108.jpg']

In [8]:
df.iloc[i]['most_likely']['response']

['4', 'Downtown', 'No', 'Highway', 'Motorcycle', 'Yes', 'Yes', 'No']

In [9]:
df.iloc[i]['question']

['How many people are riding their bicycle?\nAnswer the question using a single word or phrase.\n',
 'What part of town is this?\nAnswer the question using a single word or phrase.\n',
 'Is this area rural?\nAnswer the question using a single word or phrase.\n',
 'What kind of road are the people on?\nAnswer the question using a single word or phrase.\n',
 'What is the person riding on?\nAnswer the question using a single word or phrase.\n',
 'Is the road busy?\nAnswer the question using a single word or phrase.\n',
 'Is there a pickup truck in the picture?\nAnswer the question using a single word or phrase.\n',
 'Is there an umbrella?\nAnswer the question using a single word or phrase.\n']

In [10]:
df.iloc[i]['answers']

[{'answer': ['5',
   'expressway',
   'no',
   'freeway',
   'motorbike',
   'yes',
   'no',
   'no'],
  'answer_confidence': ['yes',
   'yes',
   'yes',
   'maybe',
   'yes',
   'yes',
   'yes',
   'yes'],
  'answer_id': tensor([1, 1, 1, 1, 1, 1, 1, 1])},
 {'answer': ['0', 'highway', 'no', 'freeway', 'bikes', 'yes', 'no', 'no'],
  'answer_confidence': ['yes',
   'maybe',
   'yes',
   'yes',
   'yes',
   'yes',
   'yes',
   'yes'],
  'answer_id': tensor([2, 2, 2, 2, 2, 2, 2, 2])},
 {'answer': ['5', 'highway', 'no', 'highway', 'motorcycle', 'yes', 'no', 'no'],
  'answer_confidence': ['yes',
   'yes',
   'yes',
   'maybe',
   'yes',
   'yes',
   'yes',
   'maybe'],
  'answer_id': tensor([3, 3, 3, 3, 3, 3, 3, 3])},
 {'answer': ['4',
   'downtown',
   'no',
   'freeway',
   'motorcycle',
   'yes',
   'no',
   'no'],
  'answer_confidence': ['maybe',
   'maybe',
   'yes',
   'yes',
   'yes',
   'yes',
   'maybe',
   'yes'],
  'answer_id': tensor([4, 4, 4, 4, 4, 4, 4, 4])},
 {'answer': ['4', 

In [11]:
embeddings=df['most_likely'].apply(lambda x: x['embedding']).to_list()

In [12]:
embeddings=np.concatenate(embeddings,axis=0)

In [13]:
embeddings.shape

(20000, 33, 4096)

In [14]:
def transpose_lines(x):
    f=pd.DataFrame(x)
    transposed=f['answer'].apply(pd.Series).T.values.tolist()
    return transposed

df['answers']=df['answers'].apply(transpose_lines)

In [15]:
gt_label=df['answers'].to_list()
gt_label=np.concatenate(gt_label,axis=0).tolist()

In [16]:
responses=df['most_likely'].apply(lambda x: x['response']).to_list()
rsps=[]
for x in responses:
    rsps+=x

In [17]:
rsps[0]

'4'

In [18]:
judge=VQAEval(gt_label,rsps)
judge.evaluate()
print(judge.evalQA)
print(judge.accuracy['overall'])

computing accuracy


  0%|          | 0/20000 [00:00<?, ?it/s]

100%|██████████| 20000/20000 [00:06<00:00, 3014.74it/s]

Done computing accuracy
{0: 100, 1: 33.33, 2: 100, 3: 100, 4: 100, 5: 100, 6: 0.0, 7: 100, 8: 100, 9: 33.33, 10: 100, 11: 100, 12: 100, 13: 100, 14: 100, 15: 100, 16: 100, 17: 100, 18: 33.33, 19: 0.0, 20: 0.0, 21: 100, 22: 33.33, 23: 0.0, 24: 0.0, 25: 100, 26: 100, 27: 100, 28: 100, 29: 100, 30: 100, 31: 33.33, 32: 100, 33: 100, 34: 33.33, 35: 0.0, 36: 100, 37: 100, 38: 100, 39: 100, 40: 100, 41: 100, 42: 100, 43: 100, 44: 100, 45: 100, 46: 100, 47: 100, 48: 100, 49: 100, 50: 0.0, 51: 100, 52: 100, 53: 0.0, 54: 100, 55: 100, 56: 100, 57: 100, 58: 0.0, 59: 0.0, 60: 100, 61: 100, 62: 0.0, 63: 100, 64: 100, 65: 100, 66: 100, 67: 100, 68: 100, 69: 100, 70: 100, 71: 66.67, 72: 100, 73: 100, 74: 100, 75: 0.0, 76: 100, 77: 33.33, 78: 0.0, 79: 0.0, 80: 66.67, 81: 100, 82: 0.0, 83: 0.0, 84: 0.0, 85: 100, 86: 100, 87: 100, 88: 0.0, 89: 33.33, 90: 100, 91: 100, 92: 100, 93: 100, 94: 100, 95: 100, 96: 100, 97: 100, 98: 33.33, 99: 100, 100: 33.33, 101: 100, 102: 100, 103: 66.67, 104: 100, 105: 33.3

In [19]:
gt_scores=[judge.evalQA[i] for i in range(len(judge.evalQA))]

In [20]:
is_correct=[]
threshold=30
for i,x in enumerate(gt_scores):
    if x >=threshold:
        is_correct.append(1)
    else:
        is_correct.append(0)
is_correct=np.array(is_correct)

In [21]:
is_correct.shape

(20000,)

In [22]:
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 20000 responses. 16833 answer correctly and 3167 answer wrongly


In [23]:
length=len(is_correct)
train_rate=0.8

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index

validset_len = 500
# exclude validation samples.
train_idxs = train_val_idxs[:- validset_len]  # trainset index
val_idxs = train_val_idxs[- validset_len:]  # validation index
hall_label_test = []
hall_label_wild = []
hall_label_val = []

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
hall_label_test = is_correct[test_idxs]
hall_label_wild = is_correct[train_idxs]
hall_label_val = is_correct[val_idxs]

print(f'trainset length:{len(hall_label_wild)}')
print(f'validset length:{len(hall_label_val)}')
print(f'testset length:{len(hall_label_test)}')
print(f'trainset postive: {sum(hall_label_wild==1)} negative: {sum(hall_label_wild==0)}')
print(f'valset postive: {sum(hall_label_val==1)} negative: {sum(hall_label_val==0)}')
print(
    f'testset postive: {sum(hall_label_test==1)} negative: {sum(hall_label_test==0)}')

trainset length:15500
validset length:500
testset length:4000
trainset postive: 13009 negative: 2491
valset postive: 430 negative: 70
testset postive: 3394 negative: 606


In [24]:
all_embeddings = embeddings
all_embeddings.shape

(20000, 33, 4096)

In [25]:
embedding_train = all_embeddings[train_idxs]
embedding_val = all_embeddings[val_idxs]
embedding_test = all_embeddings[test_idxs]

In [26]:
%load_ext autoreload
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCADiscriminator
from ml_utils.metrics import auroc,get_best_split_from_scores

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Grid Search for Best 'n_components' & 'layer'

In [36]:
def evaluator(n_components,i_layer,X,y):
    discriminator=PCADiscriminator(embedding_train[:,i_layer,:],n_components,kernel='linear')
    scores=discriminator.get_score2(X[:,i_layer,:])
    split=discriminator.get_best_split(scores,y)
    y_pred=scores>split
    return accuracy_score(y,y_pred)
    #return balanced_accuracy_score(y,y_pred)
    #return auroc(scores,y)
    #split=discriminator.get_best_split(y)
    #y_preds=(scores>split)
    #return accuracy_score(y,y_preds)

In [ ]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,2),
    'layer':range(10,embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,hall_label_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

found best n_components:1,layer:10 Score: 0.488
found best n_components:1,layer:11 Score: 0.498
found best n_components:1,layer:16 Score: 0.528
found best n_components:1,layer:17 Score: 0.544


: 

In [29]:
best_paras

{'n_components': 1, 'layer': 19, 'best_score': np.float64(0.6508305647840531)}

In [30]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [34]:
discriminator=PCADiscriminator(embedding_train[:,best_layer,:],best_paras['n_components'])
scores=discriminator.get_score2(embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(scores,hall_label_val)

In [35]:
scores=discriminator.get_score2(embedding_train[:,best_layer,:])
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(hall_label_wild,y_train)} AUROC:{roc_auc_score(hall_label_wild,scores)}')

train set acc: 0.5076774193548387 AUROC:0.6178701623947526


In [32]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_test[:,best_layer,:])
scores=discriminator.get_score()
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(hall_label_test,preds)} AUROC:{roc_auc_score(hall_label_test,scores)}')

test set acc: 0.44325 AUROC:0.6673202295021967


Train Linear Prob to Get Better Results

In [33]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=hall_label_test

In [34]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [35]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} f1:{f1_score(y_test,pred_test)}')

test set acc: 0.6605 f1:0.7707629979743417
